In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/text_to_gloss_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/text_to_gloss_model")

def generate_gloss(input_text):
    input_text = "translate: " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    outputs = model.generate(
        **inputs,
        max_length=50,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    gloss = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return gloss

test_text = "I'm going to bake a cake for my sister's birthday"
print("Generated Gloss:", generate_gloss(test_text))


Generated Gloss: bake cake sister birthday


In [ ]:
import spacy
import nltk
from transformers import GPT2LMHeadModel, GPT2Tokenizer, T5Tokenizer, T5ForConditionalGeneration
from nltk.translate.bleu_score import SmoothingFunction

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Define custom translation rules (expanded)
asl_dictionary = {
    "is": "", "are": "", "am": "", "was": "", "were": "", "be": "", "being": "", "been": "",
    "the": "", "a": "", "an": "",
    "do": "", "does": "", "did": "", "to": "",
    "today": "TODAY", "tomorrow": "TOMORROW", "yesterday": "YESTERDAY", "now": "NOW",
    "i": "ME", "me": "ME", "my": "MY", "mine": "MY", "you": "YOU", "your": "YOUR",
    "yours": "YOUR", "he": "HE", "him": "HE", "his": "HIS", "she": "SHE", "her": "HER",
    "it": "IT", "its": "ITS", "we": "WE", "us": "WE", "our": "OUR", "they": "THEY",
    "them": "THEY", "their": "THEIR",
    "can": "CAN", "can't": "CANNOT", "could": "CAN", "should": "SHOULD", "would": "WILL",
    "will": "WILL", "shall": "WILL", "must": "MUST", "might": "MAY", "may": "MAY",
    "won't": "WILL-NOT", "don't": "NOT", "doesn't": "NOT", "didn't": "NOT",
    "have": "HAVE", "need": "NEED", "want": "WANT", "know": "KNOW", "like": "LIKE",
    "eat": "EAT", "drink": "DRINK", "go": "GO", "come": "COME", "see": "SEE", "look": "LOOK",
    "watch": "WATCH", "make": "MAKE", "work": "WORK", "play": "PLAY", "learn": "LEARN",
    "teach": "TEACH", "think": "THINK", "help": "HELP",
    "and": "PLUS", "or": "OR", "but": "BUT", "because": "WHY", "if": "SUPPOSE",
    "then": "THEN", "so": "SO", "who": "WHO", "what": "WHAT", "where": "WHERE",
    "when": "WHEN", "why": "WHY", "how": "HOW", "which": "WHICH",
    "not": "NOT", "no": "NO", "none": "NONE", "never": "NEVER", "nothing": "NOTHING",
    "big": "BIG", "small": "SMALL", "happy": "HAPPY", "sad": "SAD", "angry": "ANGRY",
    "fast": "FAST", "slow": "SLOW", "easy": "EASY", "difficult": "DIFFICULT",
    "good": "GOOD", "bad": "BAD", "hot": "HOT", "cold": "COLD", "as soon as": "WHEN",
    "thank you": "THANK-YOU", "you're welcome": "WELCOME", "how are you": "HOW YOU",
    "good morning": "MORNING GOOD", "good night": "NIGHT GOOD"
}

# Verb tense mappings (adjusted)
verb_tense_mapping = {
    "VBD": "PAST", "VBG": "", "VBZ": "", "VBP": "", "VB": "",
    "AUX_VBD": "PAST", "AUX_VBN": "FINISH", "AUX_VB": "WILL",
    "MD_CONDITIONAL": "SUPPOSE", "AUX_VBG_FUTURE": "WILL", "AUX_VBD_VBG": "PAST",
    "MD_PAST": "WILL", "AUX_DO": "", "AUX_BE": ""
}

# Function to filter out unnecessary "PAST" repetitions
def handle_verb_tense(asl_gloss, tense_marker, verb):
    if tense_marker and (not asl_gloss or asl_gloss[-1] != tense_marker):
        asl_gloss.append(tense_marker)
    asl_gloss.append(verb)

def translate_to_asl_gloss(sentence):
    doc = nlp(sentence.lower())
    asl_gloss = []
    used_past_tense = False  # Track if past tense is already used

    for token in doc:
        pos = token.tag_  # Part-of-speech tag
        dep = token.dep_  # Dependency

        # Handle negation
        if dep == "neg":
            asl_gloss.append("NOT")
            continue

        # Custom translation from dictionary
        if token.text in asl_dictionary:
            translation = asl_dictionary[token.text]
            if translation:
                asl_gloss.append(translation)

        # Handle verb tenses
        elif pos in verb_tense_mapping:
            base_verb = token.lemma_.upper()
            tense_marker = verb_tense_mapping.get(pos, "")
            if tense_marker == "PAST" and used_past_tense:  # Prevent repeating "PAST"
                asl_gloss.append(base_verb)
            else:
                handle_verb_tense(asl_gloss, tense_marker, base_verb)
                if tense_marker == "PAST":
                    used_past_tense = True  # Mark that "PAST" has been used

        # Handle subordinating conjunctions
        elif dep == "mark":  # Subordinating conjunction like "when", "if"
            asl_gloss.append(token.text.upper())
            continue

        # Default case: Uppercase the word
        else:
            asl_gloss.append(token.text.upper())

    # Post-processing: Ensure time expressions move to the start if they exist
    time_indicators = {"TODAY", "YESTERDAY", "TOMORROW", "NOW"}
    for word in asl_gloss:
        if word in time_indicators:
            asl_gloss.remove(word)
            asl_gloss.insert(0, word)

    return " ".join(asl_gloss)

# Function to calculate BLEU score
def calculate_bleu(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Fine-tune the weights for different n-grams
    weights = (0.1, 0, 0, 0)  # Weights for unigrams and bigrams
    return nltk.translate.bleu_score.sentence_bleu([reference_tokens], candidate_tokens, weights=weights)

# Load T5 model and tokenizer for generating gloss
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/text_to_gloss_model")
tokenizer = T5Tokenizer.from_pretrained("/content/drive/MyDrive/text_to_gloss_model")

def generate_gloss(input_text):
    input_text = "translate: " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    outputs = model.generate(
        **inputs,
        max_length=50,
        num_beams=5,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    gloss = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return gloss

def calculate_bleu(reference, candidate):
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Use smoothing to avoid zero scores due to missing n-gram overlaps
    smoothing = SmoothingFunction().method1

    # Calculate BLEU score with smoothing
    weights = (0, 0, 0, 0)  # Adjust weights if needed
    return nltk.translate.bleu_score.sentence_bleu([reference_tokens], candidate_tokens, weights=weights, smoothing_function=smoothing)

# Function to calculate average BLEU score
def evaluate_bleu_scores(sentences, references):
    total_bleu_score = 0.0
    num_sentences = len(sentences)

    for i, sentence in enumerate(sentences):
        # Step 1: Translate English to ASL gloss using custom rules
        rule_based_gloss = translate_to_asl_gloss(sentence)

        # Step 2: Pass the rule-based gloss to the model to generate a refined gloss
        generated_gloss = generate_gloss(rule_based_gloss)
        print(f"Generated Gloss for '{sentence}': {generated_gloss}")

        # Step 3: Calculate BLEU score by comparing generated gloss with reference gloss
        bleu_score = calculate_bleu(references[i], generated_gloss)
        print(f"BLEU Score for '{sentence}': {bleu_score}")

        total_bleu_score += bleu_score

    # Compute average BLEU score
    average_bleu = total_bleu_score / num_sentences
    print(f"Average BLEU Score: {average_bleu}")

sentences = [
    "When I went to the store yesterday, I realized that I had forgotten my wallet.",
    "She will go to the party tomorrow.",
    "I am learning how to play the guitar.",
    "We had a great time at the beach last weekend.",
    "I need to buy groceries for the week.",
    "They are planning to visit their grandparents next month.",
    "Can you help me with my homework later?",
    "The dog ran across the street quickly.",
    "We are going to watch a movie tonight.",
    "My sister loves to read mystery novels.",
    "He will bring the documents to the office tomorrow morning.",
    "I was feeling sick, so I stayed home from work.",
    "The weather is nice, so let's go for a walk.",
    "She forgot her umbrella at the restaurant.",
    "I will call you when I arrive at the airport.",
    "The children are playing in the backyard.",
    "We need to finish the project by tomorrow.",
    "The concert was amazing, and I had a great time.",
    "Please send me the email once you are done.",
    "I can't find my keys anywhere.",
    "He is thinking about changing his job.",
    "We have to leave early to avoid traffic.",
    "She was excited to see her favorite band perform.",
    "They will celebrate their anniversary next weekend.",
    "I want to learn how to cook new recipes.",
    "She invited all her friends to her birthday party.",
    "We visited the museum and learned a lot about history.",
    "He is practicing for the big game tomorrow.",
    "The store opens at 9 AM every day.",
    "I will pick you up from the bus station.",
    "She likes to paint landscapes in her free time.",
    "We are going on a road trip this summer.",
    "I enjoy watching documentaries about nature.",
    "They plan to renovate their house next year.",
    "The meeting is scheduled for 3 PM this afternoon.",
    "I have to finish my report before the deadline.",
    "She was surprised by the unexpected gift.",
    "We had dinner at a nice Italian restaurant.",
    "The book was so interesting that I finished it in one day.",
    "He forgot to set the alarm last night.",
    "She will start her new job on Monday.",
    "The cat jumped onto the windowsill to look outside.",
    "I bought a new phone because my old one broke.",
    "They are organizing a charity event next month.",
    "The students are preparing for their final exams.",
    "She was happy to meet her old friend after so many years.",
    "The plane landed safely at the airport.",
    "I will send the package to you tomorrow.",
    "The kids are building a sandcastle at the beach.",
    "The car broke down on the highway last night.",
    "He always drinks coffee in the morning.",
    "They decided to adopt a puppy from the shelter.",
    "I am looking forward to the concert next week.",
    "The train arrives at the station every hour.",
    "She forgot to bring her lunch to work.",
    "I need to call my parents tonight.",
    "We are planning to go hiking this weekend.",
    "The baby started walking last month.",
    "He loves playing basketball with his friends.",
    "She is learning how to swim at the local pool.",
    "I bought a new laptop for my work.",
    "They want to travel to Japan next year.",
    "I have a doctor's appointment tomorrow.",
    "The team won the championship last season.",
    "She likes to bake cookies on the weekend.",
    "I need to renew my driver's license.",
    "The restaurant serves delicious Italian food.",
    "They went fishing at the lake yesterday.",
    "I have to finish my homework before dinner.",
    "She was late because her alarm did not go off.",
    "They are building a new park near my house.",
    "I enjoy reading books about science fiction.",
    "We will visit our cousins during the holidays.",
    "The weather forecast says it will rain tomorrow.",
    "She is preparing for her final exams.",
    "The store has a big sale on shoes this week.",
    "I want to learn how to play the piano.",
    "The dog was barking all night.",
    "He is fixing his bike in the garage.",
    "They celebrated their victory at the restaurant.",
    "I forgot to charge my phone last night.",
    "She is writing a letter to her friend.",
    "The movie starts at 7 PM.",
    "I need to clean my room this afternoon.",
    "They are waiting for the bus to arrive.",
    "She is practicing her dance routine.",
    "I found my old photos in the attic.",
    "They will move to a new house next month.",
    "We are attending a wedding this Saturday.",
    "The book tells a story about a brave hero.",
    "He works at the bank downtown.",
    "I have to wake up early tomorrow.",
    "They are learning about history in school.",
    "She is cooking dinner for her family.",
    "I need to pack my bag for the trip.",
    "They were singing songs around the campfire.",
    "She is planning to buy a new car.",
    "The kids are playing hide and seek.",
    "I am saving money to buy a new bike."
]

references = [
    "YESTERDAY WHEN ME PAST GO STORE REALIZE PAST FORGET MY WALLET",
    "TOMORROW SHE WILL GO PARTY",
    "LEARN PLAY GUITAR",
    "PAST HAVE GREAT TIME BEACH LAST WEEKEND",
    "NEED BUY GROCERIES WEEK",
    "NEXT MONTH THEY PLAN VISIT GRANDPARENTS",
    "LATER CAN YOU HELP ME HOMEWORK",
    "DOG RUN FAST STREET",
    "TONIGHT WE GO WATCH MOVIE",
    "MY SISTER LOVE READ MYSTERY BOOK",
    "TOMORROW MORNING HE WILL BRING DOCUMENTS OFFICE",
    "FEEL SICK STAY HOME WORK",
    "NICE WEATHER LET US GO WALK",
    "SHE FORGET UMBRELLA RESTAURANT",
    "ARRIVE AIRPORT I WILL CALL YOU",
    "CHILDREN PLAY BACKYARD",
    "TOMORROW NEED FINISH PROJECT",
    "CONCERT AMAZING GREAT TIME",
    "DONE EMAIL PLEASE SEND ME",
    "CANNOT FIND MY KEYS",
    "HE THINK CHANGE JOB",
    "LEAVE EARLY AVOID TRAFFIC",
    "EXCITED SEE FAVORITE BAND",
    "NEXT WEEKEND THEY CELEBRATE ANNIVERSARY",
    "LEARN COOK NEW RECIPES WANT",
    "SHE INVITE FRIENDS BIRTHDAY PARTY",
    "VISIT MUSEUM LEARN HISTORY",
    "HE PRACTICE GAME TOMORROW",
    "STORE OPEN 9 AM EVERY DAY",
    "I WILL PICK YOU BUS STATION",
    "SHE LIKE PAINT LANDSCAPE FREE TIME",
    "SUMMER WE GO ROAD TRIP",
    "ENJOY WATCH DOCUMENTARIES NATURE",
    "NEXT YEAR THEY RENOVATE HOUSE",
    "MEETING SCHEDULE 3 PM AFTERNOON",
    "BEFORE DEADLINE FINISH REPORT NEED",
    "SURPRISE UNEXPECTED GIFT",
    "DINNER ITALIAN RESTAURANT",
    "INTERESTING BOOK FINISH ONE DAY",
    "LAST NIGHT FORGET SET ALARM",
    "MONDAY SHE START NEW JOB",
    "CAT JUMP WINDOWSILL LOOK OUTSIDE",
    "BUY NEW PHONE OLD BREAK",
    "NEXT MONTH THEY ORGANIZE CHARITY EVENT",
    "STUDENT PREPARE FINAL EXAM",
    "HAPPY MEET OLD FRIEND",
    "PLANE LAND SAFE AIRPORT",
    "TOMORROW SEND PACKAGE YOU",
    "KIDS BUILD SANDCASTLE BEACH",
    "LAST NIGHT CAR BREAK HIGHWAY",
    "MORNING HE ALWAYS DRINK COFFEE",
    "THEY DECIDE ADOPT PUPPY SHELTER",
    "NEXT WEEK CONCERT I LOOK FORWARD",
    "TRAIN ARRIVE STATION EVERY HOUR",
    "SHE FORGET BRING LUNCH WORK",
    "TONIGHT NEED CALL PARENTS",
    "THIS WEEKEND WE PLAN GO HIKE",
    "LAST MONTH BABY START WALK",
    "HE LOVE PLAY BASKETBALL FRIENDS",
    "SHE LEARN SWIM LOCAL POOL",
    "BUY NEW LAPTOP WORK",
    "NEXT YEAR THEY WANT TRAVEL JAPAN",
    "TOMORROW DOCTOR APPOINTMENT HAVE",
    "LAST SEASON TEAM WIN CHAMPIONSHIP",
    "WEEKEND SHE LIKE BAKE COOKIES",
    "NEED RENEW DRIVER LICENSE",
    "RESTAURANT SERVE DELICIOUS ITALIAN FOOD",
    "YESTERDAY THEY GO FISHING LAKE",
    "BEFORE DINNER NEED FINISH HOMEWORK",
    "ALARM NOT GO OFF SHE LATE",
    "NEAR HOUSE NEW PARK BUILD",
    "ENJOY READ BOOK SCIENCE FICTION",
    "HOLIDAY WE WILL VISIT COUSINS",
    "TOMORROW WEATHER FORECAST RAIN",
    "SHE PREPARE FINAL EXAM",
    "THIS WEEK STORE BIG SALE SHOES",
    "WANT LEARN PLAY PIANO",
    "DOG BARK ALL NIGHT",
    "HE FIX BIKE GARAGE",
    "THEY CELEBRATE VICTORY RESTAURANT",
    "LAST NIGHT FORGET CHARGE PHONE",
    "SHE WRITE LETTER FRIEND",
    "MOVIE START 7 PM",
    "AFTERNOON NEED CLEAN ROOM",
    "THEY WAIT BUS ARRIVE",
    "SHE PRACTICE DANCE ROUTINE",
    "FIND OLD PHOTO ATTIC",
    "NEXT MONTH THEY WILL MOVE NEW HOUSE",
    "THIS SATURDAY WE ATTEND WEDDING",
    "BOOK STORY BRAVE HERO",
    "HE WORK BANK DOWNTOWN",
    "TOMORROW NEED WAKE UP EARLY",
    "SCHOOL THEY LEARN HISTORY",
    "SHE COOK DINNER FAMILY",
    "NEED PACK BAG TRIP",
    "THEY SING SONG CAMPFIRE",
    "SHE PLAN BUY NEW CAR",
    "KIDS PLAY HIDE SEEK",
    "SAVE MONEY BUY NEW BIKE"
]


# Evaluate BLEU scores and get the average
evaluate_bleu_scores(sentences, references)



Generated Gloss for 'When I went to the store yesterday, I realized that I had forgotten my wallet.': YESTERDAY WHEN ME PAST GO STORE REALIZE
BLEU Score for 'When I went to the store yesterday, I realized that I had forgotten my wallet.': 0.5647181220077593
Generated Gloss for 'She will go to the party tomorrow.': TOMORGO PARTY
BLEU Score for 'She will go to the party tomorrow.': 0.22313016014842982
Generated Gloss for 'I am learning how to play the guitar.': LEARN PLAY GUITAR
BLEU Score for 'I am learning how to play the guitar.': 1.0
Generated Gloss for 'We had a great time at the beach last weekend.': PAST HAVE GREAT TIME BEACH LAST WEEKEND
BLEU Score for 'We had a great time at the beach last weekend.': 1.0
Generated Gloss for 'I need to buy groceries for the week.': NEEDBUY GROCERIES WEEK
BLEU Score for 'I need to buy groceries for the week.': 0.7165313105737893
Generated Gloss for 'They are planning to visit their grandparents next month.': PLANVISIT GRANDPARENTSNEXTMONTH
BLEU Sc

In [ ]:
import pandas as pd
import json
import numpy as np
import cv2
import os

# Define the pose and hand connections (similar to MediaPipe POSE_CONNECTIONS and HAND_CONNECTIONS)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
    (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
    (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24), (23, 24),
    (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
]

HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),   # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),   # Index
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)   # Pinky
]

# Function to generate video for a single gloss
def generate_video_for_gloss(gloss, csv_folder, output_folder):
    # Construct CSV file path based on gloss name
    csv_file = os.path.join(csv_folder, f'{gloss}.csv')

    if not os.path.exists(csv_file):
        print(f"CSV file for gloss '{gloss}' not found.")
        return None

    keypoints_df = pd.read_csv(csv_file)

    # Set video output parameters for each individual gloss video
    frame_width, frame_height = 640, 480  # Adjust based on your needs
    fps = 30  # Set the frames per second for the output video
    video_output_path = os.path.join(output_folder, f'{gloss}_video.mp4')

    # Initialize video writer for the gloss video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
    out = cv2.VideoWriter(video_output_path, fourcc, fps, (frame_width, frame_height))

    # Loop through each frame in the keypoints CSV
    for index, row in keypoints_df.iterrows():
        frame_number = row['frame']  # Assuming 'frame' column exists in the CSV
        keypoints = json.loads(row['keypoints'])  # Load the keypoints JSON for the frame

        # Create a blank image (2D canvas) to plot keypoints
        img = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255  # White background

        # Plot pose keypoints and connect them with lines (if available)
        if 'pose' in keypoints:
            pose_keypoints = keypoints['pose']
            for connection in POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx < len(pose_keypoints) and end_idx < len(pose_keypoints):
                    x_start, y_start = int(pose_keypoints[start_idx][0] * frame_width), int(pose_keypoints[start_idx][1] * frame_height)
                    x_end, y_end = int(pose_keypoints[end_idx][0] * frame_width), int(pose_keypoints[end_idx][1] * frame_height)
                    cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)  # Green lines for pose connections
            # Draw keypoints as circles
            for point in pose_keypoints:
                x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                cv2.circle(img, (x, y), 5, (255, 0, 0), -1)  # Blue circles for pose keypoints

        # Plot hand keypoints and connect them with lines (if available)
        if 'hands' in keypoints:
            hand_keypoints = keypoints['hands']
            for hand in hand_keypoints:
                for connection in HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    if start_idx < len(hand) and end_idx < len(hand):
                        x_start, y_start = int(hand[start_idx][0] * frame_width), int(hand[start_idx][1] * frame_height)
                        x_end, y_end = int(hand[end_idx][0] * frame_width), int(hand[end_idx][1] * frame_height)
                        cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 0, 255), 2)  # Red lines for hand connections
                # Draw hand keypoints as circles
                for point in hand:
                    x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # Red circles for hand keypoints

        # Write the plotted frame to the video
        out.write(img)

    # Release the video writer
    out.release()

    print(f"Video for gloss '{gloss}' saved at {video_output_path}")

    return video_output_path  # Return the path of the video

# Example usage:
# Set the gloss to process
single_gloss = "hello"  # Replace with the specific gloss you want to generate a video for

# Folder paths (update accordingly)
csv_folder = "/content/drive/MyDrive/gloss_selected"  # Folder containing the keypoints CSV files
output_folder = "/content/drive/MyDrive/gloss_selected/plots_folder"  # Folder to save output videos

# Generate video for the specified gloss
generate_video_for_gloss(single_gloss, csv_folder, output_folder)


Video for gloss 'hello' saved at /content/drive/MyDrive/gloss_selected/plots_folder/hello_video.mp4


'/content/drive/MyDrive/gloss_selected/plots_folder/hello_video.mp4'

example sentence videos

In [ ]:
import pandas as pd
import json
import numpy as np
import cv2
import os

# Define the pose and hand connections (similar to MediaPipe POSE_CONNECTIONS and HAND_CONNECTIONS)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
    (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
    (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24), (23, 24),
    (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
]

HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),   # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),   # Index
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)   # Pinky
]

# Function to generate video for a single gloss
def generate_video_for_gloss(gloss, csv_folder, output_folder):
    # Construct CSV file path based on gloss name
    csv_file = os.path.join(csv_folder, f'{gloss}.csv')

    if not os.path.exists(csv_file):
        print(f"CSV file for gloss '{gloss}' not found.")
        return None

    keypoints_df = pd.read_csv(csv_file)

    # Create a temporary folder to store frames for the current gloss (optional)
    plots_folder = os.path.join(output_folder, f'plots_{gloss}')
    if not os.path.exists(plots_folder):
        os.makedirs(plots_folder)

    # Set video output parameters for each individual gloss video
    frame_width, frame_height = 640, 480  # Adjust based on your needs
    fps = 30  # Set the frames per second for the output video
    temp_video_path = os.path.join(output_folder, f'{gloss}_temp_video.mp4')

    # Initialize video writer for the temporary gloss video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
    out = cv2.VideoWriter(temp_video_path, fourcc, fps, (frame_width, frame_height))

    # Loop through each frame in the keypoints CSV
    for index, row in keypoints_df.iterrows():
        frame_number = row['frame']  # Assuming 'frame' column exists in the CSV
        keypoints = json.loads(row['keypoints'])  # Load the keypoints JSON for the frame

        # Create a blank image (2D canvas) to plot keypoints
        img = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255  # White background

        # Plot pose keypoints and connect them with lines (if available)
        if 'pose' in keypoints:
            pose_keypoints = keypoints['pose']
            for connection in POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx < len(pose_keypoints) and end_idx < len(pose_keypoints):
                    x_start, y_start = int(pose_keypoints[start_idx][0] * frame_width), int(pose_keypoints[start_idx][1] * frame_height)
                    x_end, y_end = int(pose_keypoints[end_idx][0] * frame_width), int(pose_keypoints[end_idx][1] * frame_height)
                    cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)  # Green lines for pose connections
            # Draw keypoints as circles
            for point in pose_keypoints:
                x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                cv2.circle(img, (x, y), 5, (255, 0, 0), -1)  # Blue circles for pose keypoints

        # Plot hand keypoints and connect them with lines (if available)
        if 'hands' in keypoints:
            hand_keypoints = keypoints['hands']
            for hand in hand_keypoints:
                for connection in HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    if start_idx < len(hand) and end_idx < len(hand):
                        x_start, y_start = int(hand[start_idx][0] * frame_width), int(hand[start_idx][1] * frame_height)
                        x_end, y_end = int(hand[end_idx][0] * frame_width), int(hand[end_idx][1] * frame_height)
                        cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 0, 255), 2)  # Red lines for hand connections
                # Draw hand keypoints as circles
                for point in hand:
                    x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # Red circles for hand keypoints

        # Write the plotted frame to the video
        out.write(img)

    # Release the video writer
    out.release()

    return temp_video_path  # Return the path of the temporary video

# Function to append multiple videos into one
def append_videos(video_paths, output_path):
    if len(video_paths) == 0:
        print("No videos to append.")
        return

    # Load the first video to get its properties
    first_video = cv2.VideoCapture(video_paths[0])
    frame_width = int(first_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(first_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = first_video.get(cv2.CAP_PROP_FPS)
    first_video.release()

    # Initialize the final video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Append each video
    for video_path in video_paths:
        video = cv2.VideoCapture(video_path)
        while True:
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)
        video.release()

    # Release the final video writer
    out.release()

    print(f"Final video saved at {output_path}")

# Example usage:
# Assuming the generated gloss is a sentence with multiple words like "hello world"
generated_gloss = "ME LIVE c h e n n a i"
generated_gloss = generated_gloss.lower()

# Folder paths (update accordingly)
csv_folder = "/content/drive/MyDrive/gloss_selected"  # Folder containing the keypoints CSV files
output_folder = "/content/drive/MyDrive/gloss_selected/plots_folder"  # Folder to save output videos and plots
final_output_video = os.path.join(output_folder, 'final_output_video.mp4')

# Split the gloss into individual words (glosses)
glosses = generated_gloss.split()

# Generate videos for each gloss and collect the temporary video paths
video_paths = []
for gloss in glosses:
    video_path = generate_video_for_gloss(gloss, csv_folder, output_folder)
    if video_path:
        video_paths.append(video_path)

# Append all the videos into a single video
append_videos(video_paths, final_output_video)


CSV file for gloss 'c' not found.
Final video saved at /content/drive/MyDrive/gloss_selected/plots_folder/final_output_video.mp4


In [ ]:
import pandas as pd
import json
import numpy as np
import cv2
import os

# Define the pose and hand connections (similar to MediaPipe POSE_CONNECTIONS and HAND_CONNECTIONS)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
    (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
    (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24), (23, 24),
    (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
]

HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),   # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),   # Index
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)   # Pinky
]

# Function to generate video for a single gloss
def generate_video_for_gloss(gloss, csv_folder, output_folder):
    # Construct CSV file path based on gloss name
    csv_file = os.path.join(csv_folder, f'{gloss}.csv')

    if not os.path.exists(csv_file):
        print(f"CSV file for gloss '{gloss}' not found.")
        return None

    keypoints_df = pd.read_csv(csv_file)

    # Create a temporary folder to store frames for the current gloss (optional)
    plots_folder = os.path.join(output_folder, f'plots_{gloss}')
    if not os.path.exists(plots_folder):
        os.makedirs(plots_folder)

    # Set video output parameters for each individual gloss video
    frame_width, frame_height = 640, 480  # Adjust based on your needs
    fps = 30  # Set the frames per second for the output video
    temp_video_path = os.path.join(output_folder, f'{gloss}_temp_video.mp4')

    # Initialize video writer for the temporary gloss video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
    out = cv2.VideoWriter(temp_video_path, fourcc, fps, (frame_width, frame_height))

    # Loop through each frame in the keypoints CSV
    for index, row in keypoints_df.iterrows():
        frame_number = row['frame']  # Assuming 'frame' column exists in the CSV
        keypoints = json.loads(row['keypoints'])  # Load the keypoints JSON for the frame

        # Create a blank image (2D canvas) to plot keypoints
        img = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255  # White background

        # Plot pose keypoints and connect them with lines (if available)
        if 'pose' in keypoints:
            pose_keypoints = keypoints['pose']
            for connection in POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx < len(pose_keypoints) and end_idx < len(pose_keypoints):
                    x_start, y_start = int(pose_keypoints[start_idx][0] * frame_width), int(pose_keypoints[start_idx][1] * frame_height)
                    x_end, y_end = int(pose_keypoints[end_idx][0] * frame_width), int(pose_keypoints[end_idx][1] * frame_height)
                    cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)  # Green lines for pose connections
            # Draw keypoints as circles
            for point in pose_keypoints:
                x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                cv2.circle(img, (x, y), 5, (255, 0, 0), -1)  # Blue circles for pose keypoints

        # Plot hand keypoints and connect them with lines (if available)
        if 'hands' in keypoints:
            hand_keypoints = keypoints['hands']
            for hand in hand_keypoints:
                for connection in HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    if start_idx < len(hand) and end_idx < len(hand):
                        x_start, y_start = int(hand[start_idx][0] * frame_width), int(hand[start_idx][1] * frame_height)
                        x_end, y_end = int(hand[end_idx][0] * frame_width), int(hand[end_idx][1] * frame_height)
                        cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 0, 255), 2)  # Red lines for hand connections
                # Draw hand keypoints as circles
                for point in hand:
                    x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # Red circles for hand keypoints

        # Write the plotted frame to the video
        out.write(img)

    # Release the video writer
    out.release()

    return temp_video_path  # Return the path of the temporary video

# Function to append multiple videos into one
def append_videos(video_paths, output_path):
    if len(video_paths) == 0:
        print("No videos to append.")
        return

    # Load the first video to get its properties
    first_video = cv2.VideoCapture(video_paths[0])
    frame_width = int(first_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(first_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = first_video.get(cv2.CAP_PROP_FPS)
    first_video.release()

    # Initialize the final video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Append each video
    for video_path in video_paths:
        video = cv2.VideoCapture(video_path)
        while True:
            ret, frame = video.read()
            if not ret:
                break
            out.write(frame)
        video.release()

    # Release the final video writer
    out.release()

    print(f"Final video saved at {output_path}")

# Example usage:
# Assuming the generated gloss is a sentence with multiple words like "hello world"
generated_gloss = "help me need"
generated_gloss = generated_gloss.lower()

# Folder paths (update accordingly)
csv_folder = "/content/drive/MyDrive/gloss_selected"  # Folder containing the keypoints CSV files
output_folder = "/content/drive/MyDrive/gloss_selected/plots_folder"  # Folder to save output videos and plots
final_output_video = os.path.join(output_folder, 'final_output_video.mp4')

# Split the gloss into individual words (glosses)
glosses = generated_gloss.split()

# Generate videos for each gloss and collect the temporary video paths
video_paths = []
for gloss in glosses:
    video_path = generate_video_for_gloss(gloss, csv_folder, output_folder)
    if video_path:
        video_paths.append(video_path)

# Append all the videos into a single video
append_videos(video_paths, final_output_video)


Final video saved at /content/drive/MyDrive/gloss_selected/plots_folder/final_output_video.mp4


In [ ]:
import pandas as pd
import json
import numpy as np
import cv2
import os

# Define the pose and hand connections (similar to MediaPipe POSE_CONNECTIONS and HAND_CONNECTIONS)
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), (9, 10),
    (11, 12), (11, 13), (13, 15), (15, 17), (15, 19), (15, 21), (12, 14),
    (14, 16), (16, 18), (16, 20), (16, 22), (11, 23), (12, 24), (23, 24),
    (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
]

HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),   # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),   # Index
    (0, 9), (9, 10), (10, 11), (11, 12),  # Middle
    (0, 13), (13, 14), (14, 15), (15, 16),  # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)   # Pinky
]

# Function to generate video for a single word gloss
def generate_video_for_gloss(gloss, csv_folder):
    # Construct CSV file path based on gloss name
    csv_file = os.path.join(csv_folder, f'{gloss}.csv')

    if not os.path.exists(csv_file):
        print(f"CSV file for gloss '{gloss}' not found.")
        return None

    keypoints_df = pd.read_csv(csv_file)

    # Set video output parameters for each individual gloss video
    frame_width, frame_height = 640, 480  # Adjust based on your needs
    fps = 30  # Set the frames per second for the output video
    temp_video_path = f'{gloss}_temp.mp4'

    # Initialize video writer for the gloss video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec
    out = cv2.VideoWriter(temp_video_path, fourcc, fps, (frame_width, frame_height))

    # Loop through each frame in the keypoints CSV
    for index, row in keypoints_df.iterrows():
        frame_number = row['frame']  # Assuming 'frame' column exists in the CSV
        keypoints = json.loads(row['keypoints'])  # Load the keypoints JSON for the frame

        # Create a blank image (2D canvas) to plot keypoints
        img = np.ones((frame_height, frame_width, 3), dtype=np.uint8) * 255  # White background

        # Plot pose keypoints and connect them with lines (if available)
        if 'pose' in keypoints:
            pose_keypoints = keypoints['pose']
            for connection in POSE_CONNECTIONS:
                start_idx, end_idx = connection
                if start_idx < len(pose_keypoints) and end_idx < len(pose_keypoints):
                    x_start, y_start = int(pose_keypoints[start_idx][0] * frame_width), int(pose_keypoints[start_idx][1] * frame_height)
                    x_end, y_end = int(pose_keypoints[end_idx][0] * frame_width), int(pose_keypoints[end_idx][1] * frame_height)
                    cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)  # Green lines for pose connections
            # Draw keypoints as circles
            for point in pose_keypoints:
                x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                cv2.circle(img, (x, y), 5, (255, 0, 0), -1)  # Blue circles for pose keypoints

        # Plot hand keypoints and connect them with lines (if available)
        if 'hands' in keypoints:
            hand_keypoints = keypoints['hands']
            for hand in hand_keypoints:
                for connection in HAND_CONNECTIONS:
                    start_idx, end_idx = connection
                    if start_idx < len(hand) and end_idx < len(hand):
                        x_start, y_start = int(hand[start_idx][0] * frame_width), int(hand[start_idx][1] * frame_height)
                        x_end, y_end = int(hand[end_idx][0] * frame_width), int(hand[end_idx][1] * frame_height)
                        cv2.line(img, (x_start, y_start), (x_end, y_end), (0, 0, 255), 2)  # Red lines for hand connections
                # Draw hand keypoints as circles
                for point in hand:
                    x, y = int(point[0] * frame_width), int(point[1] * frame_height)
                    cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # Red circles for hand keypoints

        # Write the plotted frame to the video
        out.write(img)

    # Release the video writer
    out.release()

    return temp_video_path

# Function to combine videos for the full sentence gloss
def generate_combined_video(sentence_gloss, csv_folder, output_folder):
    words = sentence_gloss.split()  # Split sentence into individual words
    temp_videos = []

    # Generate video for each word and store in temporary list
    for word in words:
        video_path = generate_video_for_gloss(word, csv_folder)
        if video_path:
            temp_videos.append(video_path)

    # Set parameters for the final combined video
    frame_width, frame_height = 640, 480
    fps = 30
    final_output_path = os.path.join(output_folder, f'{sentence_gloss.replace(" ", "_")}_video.mp4')

    # Initialize video writer for the combined video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(final_output_path, fourcc, fps, (frame_width, frame_height))

    # Concatenate all temporary videos into the final output
    for video in temp_videos:
        cap = cv2.VideoCapture(video)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
        cap.release()

    # Release the final video writer
    out.release()

    # Remove temporary videos
    for video in temp_videos:
        os.remove(video)

    print(f"Final video for sentence gloss '{sentence_gloss}' saved at {final_output_path}")

    return final_output_path

# Example usage:
# Set the sentence gloss to process
sentence_gloss = "hello world"  # Replace with the specific sentence gloss you want to generate a video for

# Folder paths (update accordingly)
csv_folder = "/content/drive/MyDrive/gloss_selected"  # Folder containing the keypoints CSV files
output_folder = "/content/drive/MyDrive/gloss_selected/plots_folder"  # Folder to save output videos

# Generate final video for the specified sentence gloss
generate_combined_video(sentence_gloss, csv_folder, output_folder)


Final video for sentence gloss 'hello world' saved at /content/drive/MyDrive/gloss_selected/plots_folder/hello_world_video.mp4


'/content/drive/MyDrive/gloss_selected/plots_folder/hello_world_video.mp4'